In [1]:
import numpy as np
import pyvista as pv
import os

In [ ]:
sim=0
data = np.load('postprocessed_data/NP_data/data.npy')
coords = np.load('postprocessed_data/NP_data/coords.npy')
coords_o = np.load('postprocessed_data/NP_data/coords_o.npy')
sdf = np.load('postprocessed_data/NP_data/sdf.npy')
sdf_o = np.load('postprocessed_data/NP_data/sdf_o.npy')
sdf_o = sdf_o[sim,:,:]

tidx = 20
data_o = data[tidx,:,:] # original

# Reshape for StructuredGrid
mesh_o = pv.StructuredGrid()
mesh_o.points = coords_o
mesh_o.dimensions = (200, 40, 40)  

# Add scalar field
mesh_o['u'] = data_o[:,0]
mesh_o['v'] = data_o[:,1]
mesh_o['w'] = data_o[:,2]
mesh_o['p'] = data_o[:,3]
# mesh_o['p'] = mesh_o['p']/np.maximum(mesh_o['p'],5e4)
mesh_o['g'] = sdf_o
mesh_o['m'] = sdf_o>0

# Create sampling box
bounds = [-1, 1, -0.5, 0.5, -0.5, 0.5]

# Extract coordinates of all points in the mesh
points = mesh_o.points

# Create a mask for points inside the box
mask = (
    (points[:, 0] >= bounds[0]) & (points[:, 0] <= bounds[1]) &
    (points[:, 1] >= bounds[2]) & (points[:, 1] <= bounds[3]) &
    (points[:, 2] >= bounds[4]) & (points[:, 2] <= bounds[5])
)

# Get the indices of the points within the box
ids = np.where(mask)[0]

# Extract point data and create a new point cloud mesh
# sampled_data = mesh_o.extract_points(ids)
sample_x = mesh_o.points[mask]
sample_u = mesh_o['u'][mask]
sample_v = mesh_o['v'][mask]
sample_w = mesh_o['w'][mask]
sample_p = mesh_o['p'][mask]
sample_data = pv.PolyData(sample_x)
sample_data['u'] = sample_u
sample_data['v'] = sample_v
sample_data['w'] = sample_w
sample_data['p'] = sample_p

# Normalize
# for scalar in mesh_o.array_names:
#     if scalar != 'g' or scalar !='m':
#         mesh_o[scalar] = mesh_o[scalar]/np.max(np.abs(mesh_o[scalar]))#*mesh_o['m'] 

# Define scalar and value ranges
scalar = 'u'
clim_dict = {
    'u': (-1,1),
    'v': (-1,1),
    'w': (-1,1),
    'p': (-1,1),
    'g': (-1,1),
    'm': (0,1),
}

# # Define the start and end points of line
# start_point = [-4, -2, 0]
# end_point = [4, 2, 0]

# # Sample the data from the dataset onto the line
# sampled_data = mesh_o.sample_over_line(start_point,end_point,resolution=20)

# Clip for vis
mesh_o = mesh_o.clip('y')

# Visualize
pl = pv.Plotter()
pl.add_mesh(mesh_o, scalars=scalar,
            # clim=clim_dict[scalar],
            opacity=0.3
            # show_edges=True
            )
pl.add_mesh(sample_data, scalars=scalar, style='points')
pl.camera_position='zx'
pl.show()


Widget(value='<iframe src="http://localhost:33135/index.html?ui=P_0x7790e9cbf5c0_25&reconnect=auto" class="pyv…

In [71]:
# Save
np.save('postprocessed_data/NP_data/doppler_x.npy',np.array(sample_x))
np.save('postprocessed_data/NP_data/doppler_u.npy',np.array(sample_u))
np.save('postprocessed_data/NP_data/doppler_v.npy',np.array(sample_v))
np.save('postprocessed_data/NP_data/doppler_w.npy',np.array(sample_w))
np.save('postprocessed_data/NP_data/doppler_p.npy',np.array(sample_p))